# 3 · A Busca pelo Equilíbrio

**Registro observacional associado ao livro**  
*Descobrindo o Caos nos Números — Como a ordem emerge quando mudamos a forma de observar*  
© Alvaro Costa, 2025  

Este notebook faz parte de uma sequência canônica de registros computacionais.  
Ele não introduz hipóteses, conjecturas ou modelos interpretativos novos.

Seu objetivo é exclusivamente **registrar** o comportamento de estruturas aritméticas sob um regime  
de observação explícito, determinístico e reproduzível.

A leitura conceitual completa encontra-se no livro. Este notebook documenta apenas o experimento  
correspondente.

**Licença:** Creative Commons BY–NC–ND 4.0  
É permitida a leitura, execução e citação.  
Não é permitida a modificação, redistribuição adaptada ou uso comercial independente.


## 1. O Paradoxo da Dívida Crescente

No capítulo anterior, observamos o *pulso* do sistema, $\Delta_\pi(x)$.  
Vimos que, após uma breve fase inicial de euforia (valores positivos), a curva mergulha em um vale de valores  
negativos — e esse vale parece aprofundar-se à medida que $x$ cresce.

À primeira vista, isso soa paradoxal.  
Se a diferença absoluta entre Estruturadores e Estabilizadores cresce indefinidamente, o sistema não estaria cada  
vez mais **desequilibrado**?  
A “dívida” dos primos estabilizadores nunca seria paga?

Essa impressão é apenas parcial. O aparente desequilíbrio é um **efeito de escala**, e não de ruptura.

---

## 2. A Perspectiva Correta · Harmonia Relativa

Para compreender sistemas que crescem indefinidamente, não basta observar diferenças absolutas — é preciso adotar  
uma **perspectiva relativa**.

Lembremos que o número total de primos é a soma das duas forças:

$
\pi(x) = \pi_S(x) + \pi_N(x)
$

Dividindo tudo por $\pi(x)$, obtemos uma identidade simples e profunda —  
a **Lei de Conservação dos Primos**:

$
1 = \frac{\pi_S(x)}{\pi(x)} + \frac{\pi_N(x)}{\pi(x)}
$

Ela nos diz que, embora as contagens absolutas variem, as frações de Estruturadores e Estabilizadores **sempre  
somam exatamente 1**. São duas faces de um mesmo equilíbrio dinâmico.

O **Teorema dos Números Primos** garante que, à medida que $x \to \infty$, ambas as frações convergem para o  
mesmo valor: $1/2$. A estrutura e a estabilidade tornam-se, em média, indistinguíveis.

Revisitando então nossa medida de tensão, $\Delta_\pi(x)$, podemos reescrevê-la em forma **normalizada**:

$
\frac{\Delta_\pi(x)}{\pi(x)} = 
\frac{\pi_N(x)}{\pi(x)} - \frac{\pi_S(x)}{\pi(x)}
$

Se cada termo tende a $1/2$, a diferença tende a **zero**.  
A “dívida” entre as duas forças pode crescer em termos absolutos, mas, *relativamente ao todo*, desaparece.  
O sistema, visto de longe, caminha inexoravelmente para o equilíbrio.

---

## 3. Laboratório Interativo · A Convergência para o Equilíbrio

A célula abaixo cria um pequeno experimento para visualizar essa convergência harmônica.  
Aumente o limite $N$ com o *slider* e observe os dois gráficos.

**O que observar:**

1. **Gráfico 1:** As curvas azul e laranja — Estruturadores e Estabilizadores — aproximam-se da linha tracejada  
de $\tfrac{1}{2}$, como se atraídas por um campo de simetria.  
2. **Gráfico 2:** A diferença relativa entre as duas forças oscila intensamente para $x$ baixos, mas essas flutuações  
se amortecem, e a curva converge suavemente para o equilíbrio em $0$.


In [1]:
# Requisitos: matplotlib, numpy, ipywidgets
# Execute no Colab ou Jupyter com o kernel correto.

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
import time

# --- Cores (para consistência) ---
CIP_BLUE   = "#0055A4"
CIP_ORANGE = "#FF7F11"

# --- Função Auxiliar para gerar primos ---
def primos_ate(n: int) -> np.ndarray:
    """Gera um array com todos os primos até n."""
    if n < 2: return np.array([], dtype=int)
    sieve = np.ones(n + 1, dtype=bool)
    sieve[:2] = False
    for i in range(2, int(np.sqrt(n)) + 1):
        if sieve[i]:
            sieve[i*i : n+1 : i] = False
    return np.flatnonzero(sieve)

# --- A Função Interativa Principal (Otimizada) ---
def plot_interactive_relative_harmony(log_N_max=4):
    
    N_max = int(10**log_N_max)
    print(f"Gerando dados até x = {N_max:,}...")
    start_time = time.time()

    # --- 1. Geração de Dados Vetorizada e Otimizada ---
    primos = primos_ate(N_max)
    
    # Cria um array de contagem cumulativa de primos (pi(x)) de forma eficiente
    pi_cumulative = np.zeros(N_max + 1, dtype=int)
    if primos.size > 0:
        pi_cumulative[primos] = 1
    pi_cumulative = np.cumsum(pi_cumulative)

    # Calcula todas as métricas de uma vez usando operações de array NumPy
    x_range = np.arange(1, N_max + 1)
    pi_x_vals = pi_cumulative[x_range]
    pi_S_vals = pi_cumulative[x_range // 2]      # pi(floor(x/2))
    pi_N_vals = pi_x_vals - pi_S_vals          # pi(x) - pi(floor(x/2))
    delta_pi_vals = pi_N_vals - pi_S_vals        # pi_N(x) - pi_S(x)
    
    # Prepara as frações e a tensão relativa, evitando divisão por zero
    pi_x_safe = np.where(pi_x_vals == 0, np.nan, pi_x_vals)
    frac_S = pi_S_vals / pi_x_safe
    frac_N = pi_N_vals / pi_x_safe
    delta_pi_ratio = delta_pi_vals / pi_x_safe
    
    end_time = time.time()
    print(f"-> Dados gerados em {end_time - start_time:.2f} segundos.")

    # --- 2. Criação dos Gráficos ---
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, axes = plt.subplots(2, 1, figsize=(12, 9), sharex=True)

    # --- Gráfico 1: A Lei da Conservação ---
    ax1 = axes[0]
    ax1.semilogx(x_range, frac_S, lw=1.5, label=r"Fração de Estruturadores: $\pi_S(x)/\pi(x)$", color=CIP_BLUE)
    ax1.semilogx(x_range, frac_N, lw=1.5, label=r"Fração de Estabilizadores: $\pi_N(x)/\pi(x)$", color=CIP_ORANGE)
    ax1.axhline(0.5, ls="--", color='black', lw=1.0, alpha=0.7, label=r"Limite de Equilíbrio (1/2)")
    ax1.set_ylim(0, 1.05) # Pequena margem no topo
    ax1.set_ylabel("Fração do Total de Primos")
    ax1.set_title("1. A Lei da Conservação: As Frações Convergem para 1/2", fontsize=14)
    ax1.legend(loc='upper right')

    # --- Gráfico 2: A Convergência da Tensão Relativa ---
    ax2 = axes[1]
    ax2.semilogx(x_range, delta_pi_ratio, linewidth=1.5, color='black', label=r"$\Delta_\pi(x)/\pi(x)$")
    ax2.axhline(0, linestyle="--", color='black', linewidth=1.2, alpha=0.7, label="Equilíbrio Perfeito (y = 0)")
    ax2.set_ylabel("Diferença Relativa")
    ax2.set_xlabel("x (escala logarítmica)")
    ax2.set_title(r"2. A Harmonia Relativa: A Tensão $\Delta_\pi(x)/\pi(x)$ Converge para Zero", fontsize=14)
    ax2.legend(loc='upper right')

    fig.tight_layout(pad=2.0)
    plt.show()

# --- Cria o Widget Interativo ---
interact(plot_interactive_relative_harmony, 
         log_N_max=widgets.IntSlider(min=2, max=7, step=1, value=4, description='Ver N até 10^', continuous_update=False));

interactive(children=(IntSlider(value=4, continuous_update=False, description='Ver N até 10^', max=7, min=2), …

### Um Sistema Quântico Disfarçado?

As oscilações de $\frac{\Delta_\pi(x)}{\pi(x)}$ lembram o comportamento de um sistema quântico relaxando em direção ao equilíbrio.  
As duas forças — Estruturadora e Estabilizadora — comportam-se como estados de um sistema de dois níveis, trocando energia até a  
simetria se restabelecer.  
A escala logarítmica atua como um tempo interno, em que o caos inicial dá lugar à ordem espectral.

Visto sob essa lente, a reta numérica não é estática — é um campo oscilatório cuja harmonia final emerge exatamente do conflito  
que a gerou.

---

> O que parecia um descompasso infinito revela-se, afinal, uma harmonia que cresce com a própria escala.  
> O caos inicial não é erro — é o preço da expansão.

---

**Nota metodológica:**
A analogia com sistemas quânticos é estritamente descritiva.  
Nenhuma estrutura física, operador quântico ou modelo dinâmico é assumido.  
O termo é usado apenas para descrever o padrão observacional de relaxamento em direção ao equilíbrio relativo.

Este equilíbrio relativo não elimina o sinal — ele o prepara.  
Nos próximos registros, o mesmo comportamento será observado sob transformações espectrais, onde a noção de harmonia deixará de  
ser apenas proporcional e passará a ser estrutural.